In [2]:
import json
import requests
import yaml
import pandas as pd
import warnings
from bs4 import BeautifulSoup
import re
import numpy as np
from shapely.geometry import Point
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#key_dict = {'key' : ''}

In [4]:
# dump the key into yaml 
with open('credentials.yml','w') as output:
    yaml.dump(key_dict, output)
    
# get the key from .yaml file
key_dict = yaml.load(open('credentials.yml'), Loader=yaml.FullLoader)  

In [5]:
key_dict = yaml.load(open("credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = key_dict["key"]
api_key = gmaps_key

### Scraping Coop locations in Zürich

In [6]:
page = requests.get("https://www.profital.ch/de/flr/81450-migros-filialen/2699")
print(page.status_code) ## prints 200 as status code
#print(page.content[:3000]) ## prints the entire webpage content
soup_1 = BeautifulSoup(page.content, "html.parser")
type(soup_1)

200


bs4.BeautifulSoup

In [7]:
soup_1.findAll('address');

In [8]:
Loc=[]
for pg in range(2,4):
    page_2 = requests.get(f"https://www.profital.ch/de/flr/81450-migros-filialen/2699?page={pg}")
    soup_2 = BeautifulSoup(page_2.content,"html.parser")
    soup_2.findAll('address')
    Loc.append(soup_2.findAll('address'))

In [9]:
Loc.append(soup_1.findAll('address'))
page_num=len(Loc)

In [10]:
Address=[]
Zip=[]
for i in range(0,page_num):
    for j in range(0,len(Loc[i])):
        Address.append(Loc[i][j].contents[0].strip())
        Zip.append(Loc[i][j].contents[2].strip().split()[0])
len(Address)

33

In [ ]:
df_Migros_Locator = pd.DataFrame(list(zip(Address,Zip)), columns=["Address", "Zip_Code"])
df_Migros_Locator

In [14]:
df_Migors_Address = pd.DataFrame()
# Combine Address and Zip Code into a single column
df_Migors_Address["address"] = df_Migros_Locator["Address"] + ', ' + df_Migros_Locator["Zip_Code"]
df_Migors_Address.head()

,address
0,"Winterthurerstrasse 2 - 4, 8006"
1,"Falkenstrasse 19 - 21, 8008"
2,"Giesshübelstrasse 65, 8045"
3,"Toblerstrasse 73, 8044"
4,"Kreuzplatz 20, 8008"


### Getting store locations from Google Maps and mapping the location to Kreis name

In [16]:
from shapely.geometry import shape, GeometryCollection, Point

In [17]:
# Function to get latitude and longitude from Google Maps API
def get_lat_lng(api_key, query):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    req = requests.get(url + "query=" + query + "&key=" + api_key)
    places_json = req.json()
    my_result = places_json.get("results", [])

    if my_result:
        location = my_result[0].get("geometry", {}).get("location", {})
        lat, lng = location.get("lat"), location.get("lng")
        return lat, lng
    else:
        return None, None

In [18]:
# Function to get kreis_name from GeoJSON data
def get_kreis_name(point, geojson_data):
    for feature in geojson_data["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]["bezeichnung"]
    return None

In [19]:
# Load GeoJSON data
with open('data/stzh.adm_stadtkreise_a.json', 'r') as f:
    geojson_data = json.load(f)

# Create a DataFrame to store the results
columns = ["address", "latitude", "longitude", "kreis_name"]
migros_df = pd.DataFrame(columns=columns)

# Iterate through addresses in the DataFrame
for index, row in df_Migors_Address.iterrows():
    address = row["address"]
    
    # Get latitude and longitude from Google Maps API
    lat, lng = get_lat_lng(api_key, address)
    
    # If coordinates are found, create a Point object
    if lat is not None and lng is not None:
        point = Point(lng, lat)
        
        # Get kreis_name from GeoJSON data
        kreis_name = get_kreis_name(point, geojson_data)
        
        # Append results to DataFrame
        migros_df = pd.concat([migros_df, pd.DataFrame([{"address": address, "latitude": lat, "longitude": lng, "kreis_name": kreis_name}])], ignore_index=True)

# Display the final DataFrame
migros_df.head()

,address,latitude,longitude,kreis_name
0,"Winterthurerstrasse 2 - 4, 8006",47.386080,8.548706,Kreis 6
1,"Falkenstrasse 19 - 21, 8008",47.365452,8.547869,Kreis 1
2,"Giesshübelstrasse 65, 8045",47.358264,8.517796,Kreis 3
3,"Toblerstrasse 73, 8044",47.379402,8.559434,Kreis 7
4,"Kreuzplatz 20, 8008",47.364816,8.554096,Kreis 7


In [20]:
migros_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   address     33 non-null     object 
 1   latitude    33 non-null     float64
 2   longitude   33 non-null     float64
 3   kreis_name  33 non-null     object 
dtypes: float64(2), object(2)
memory usage: 1.2+ KB


In [21]:
# Save DataFrame to CSV
migros_df.to_csv('migros_data.csv', index=False)